In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple
import helpers
import os
import datetime
tf.reset_default_graph()

In [2]:
lstm_size_encoder = 128
lstm_size_decoder = 256
vocab_size = 679 
embed_size = 256
encoder_cell = LSTMCell(lstm_size_encoder)
decoder_cell = LSTMCell(lstm_size_decoder)
GO = 0
PAD = 0
EOS = 1

## Load Data: English2French

In [3]:
data_dir = "en_fr"
input_file_en = os.path.join(data_dir, "A_62_952_ADD1_en.snt")
lines_en = []
with open(input_file_en, "r", encoding="utf-8") as f_en:
    for line in f_en.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_en.append(line)
        
print(lines_en[-2])
#print(("Text loaded from '%s'") % (input_file))

A revised edition of the rules of procedure has been issued under the symbol A/520/Rev.17.


In [4]:
data_dir = "en_fr"
#save_dir = "eng_fre_dataset/multiUN.en/un/text/en-fr/2009"
input_file_fr = os.path.join(data_dir, "A_62_952_ADD1_fr.snt")
lines_fr = []
with open(input_file_fr, "r", encoding="utf-8") as f_fr:
    for line in f_fr.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_fr.append(line)
        
print(lines_fr[-2])
#print(("Text loaded from '%s'") % (input_file))

Une version révisée du Règlement intérieur a été publiée sous la cote A/520/Rev.17.


In [5]:
len(lines_en), len(lines_fr)

(67, 67)

In [6]:
word_data = []
for i in range(len(lines_en)):
    word_data.append((lines_en[i],lines_fr[i]))

In [7]:
word_data[:5]

[('Sixty-second session', 'Soixante-deuxième session'),
 ('Agenda item 121', "Point 121 de l'ordre du jour"),
 ('Revitalization of the work of the General Assembly',
  "Revitalisation des travaux de l'Assemblée générale"),
 ('Report of the Ad Hoc Working Group on the Revitalization of the General Assembly',
  "Rapport du Groupe de travail spécial sur la revitalisation de l'Assemblée générale"),
 ('Addendum', 'Additif')]

In [9]:
x = [x for x, y in word_data]
y = [y for x, y in word_data]

In [10]:
x_list = []
for i in range(len(x)):
    x_text = x[i].split()
    x_list.append(x_text)
for i in range(1, len(x_list)):
    x_list[i] = set(x_list[i-1]).union(set(x_list[i]))
    x_words_set = x_list[len(x_list)-1]

In [11]:
y_list = []
for i in range(len(y)):
    y_text = y[i].split()
    y_list.append(y_text)
for i in range(1, len(y_list)):
    y_list[i] = set(y_list[i-1]).union(set(y_list[i]))
    y_words_set = y_list[len(y_list)-1]

In [12]:
word2numX = dict(zip(x_words_set, range(len(x_words_set))))
word2numY = dict(zip(y_words_set, range(len(y_words_set))))

In [13]:
num2wordX = dict(zip(word2numX.values(), word2numX.keys()))

In [14]:
# Extracting max_len
numWordsList = []
for word in x:
    Words = word.split(' ')
    num = len(Words)
    numWordsList.append(num) 
max_len = max(numWordsList)

In [15]:
step1_X = [
    word.split() for word in x
]

In [16]:
second_list = []
for i in range(len(step1_X)):
    first_list = []
    for j in range(len(step1_X[i])):
        k = word2numX[step1_X[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
x = second_list
x = np.array(x)

In [17]:
num2wordY = dict(zip(word2numY.values(), word2numY.keys()))

In [18]:
step1_Y = [
    word.split() for word in y
]

In [19]:
second_list = []
for i in range(len(step1_Y)):
    first_list = []
    for j in range(len(step1_Y[i])):
        k = word2numY[step1_Y[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
y = second_list
y = np.array(y)

In [20]:
print(len(word2numX))
print(len(word2numY))


679
720


In [21]:
print(num2wordX)

{0: '15', 1: 'In', 2: 'Hammarskjöld', 3: 'training', 4: 'found', 5: 'results', 6: 'such', 7: 'next', 8: 'subdivided', 9: 'over', 10: '1948.', 11: 'context,', 12: 'Secretary-General', 13: 'are', 14: 'committees', 15: 'raise', 16: 'III', 17: 'Swahili)', 18: 'frequently', 19: 'On', 20: 'A', 21: 'several', 22: 'catalogue', 23: 'scope', 24: 'included', 25: 'distributors;', 26: 'year', 27: 'also', 28: 'each', 29: '(www.un.org/Depts/dhl/resguide/),', 30: 'speeches', 31: 'on', 32: 'various', 33: 'daily', 34: 'would', 35: 'promptly', 36: 'conventions,', 37: 'signal,', 38: 'anniversary', 39: 'I:', 40: 'evaluation', 41: 'official', 42: 'nature', 43: 'into', 44: 'Centre', 45: 'versions', 46: 'whether', 47: 'I', 48: 'issues', 49: 'be', 50: 'support', 51: 'tour,', 52: 'subsidiary', 53: 'Revitalization', 54: 'types', 55: 'Committee', 56: 'States', 57: 'inventory/chart', 58: 'include:', 59: 'implementation', 60: 'French', 61: 'Meetings', 62: 'representatives', 63: 'new', 64: 'mostly', 65: 'issued', 66

In [22]:
print(num2wordY)

{0: '15', 1: 'Hammarskjöld', 2: 'chevauchement,', 3: 'son', 4: 'bibliothèque', 5: 'la', 6: 'Partenariat', 7: 'sous', 8: 'discours', 9: 'matière', 10: '1948.', 11: 'preuves', 12: 'portugais', 13: 'reflète', 14: 'informations', 15: 'III', 16: 'Additif', 17: 'départ', 18: 'contenu,', 19: 'cas', 20: 'avec', 21: 'pratiques,', 22: 'trente-sixième', 23: "d'articles", 24: 'mandats', 25: 'développement,', 26: 'retrait', 27: 'présidents', 28: 'concrètes', 29: 'Revitalisation', 30: 'souvent', 31: 'liste', 32: 'manière', 33: 'mise', 34: 'fournis', 35: 'anglais', 36: 'leurs', 37: "l'importance", 38: 'trois', 39: 'unième', 40: 'précise', 41: 'conventions,', 42: 'Règlement', 43: 'mesures', 44: 'répertorier', 45: 'personnes', 46: 'Annexe', 47: 'bulletins', 48: 'nature', 49: 'ressources', 50: 'Centre', 51: 'versions', 52: 'I', 53: 'commissions.', 54: "l'élaboration", 55: 'sa', 56: 'intérieur,', 57: 'États', 58: "d'orientation.", 59: 'types', 60: 'Soixante-deuxième', 61: 'progrès.', 62: 'utiliser', 63: 

In [23]:
len(x)

67

In [24]:
len(y)

67

## Embeddings: Used int type embedding lookup

In [25]:
encoder_inputs = tf.placeholder(tf.int32, (None, None), "encoder_inputs")
encoder_inputs_length = tf.placeholder(tf.int32, (None, ), "encoder_inputs_length")
decoder_inputs = tf.placeholder(tf.int32, (None, None), "decoder_inputs")
decoder_targets = tf.placeholder(tf.int32, (None, None), "decoder_targets")

In [26]:
input_embedding = tf.Variable(tf.random_uniform((vocab_size, embed_size), -1.0, 1.0), name="input_embedding")
output_embedding = tf.Variable(tf.random_uniform((len(word2numY),embed_size), -1.0, 1.0), name="output_embedding")
embedded_input = tf.nn.embedding_lookup(input_embedding, encoder_inputs)
embedded_output = tf.nn.embedding_lookup(output_embedding, decoder_inputs)

## Encoder: Bidirectional LSTM

In [27]:
((enc_fw_outputs, enc_bw_outputs),
(enc_fw_last_state, enc_bw_last_state)) = (tf.nn.bidirectional_dynamic_rnn
                                            (cell_fw=encoder_cell,
                                             cell_bw=encoder_cell,
                                             inputs=embedded_input,
                                             sequence_length=encoder_inputs_length,
                                             dtype=tf.float32, time_major=True))

In [28]:
encoder_outputs = tf.concat((enc_fw_outputs, enc_bw_outputs), 2)
enc_last_state_c = tf.concat((enc_fw_last_state.c, enc_bw_last_state.c),1)
enc_last_state_h = tf.concat((enc_fw_last_state.h, enc_bw_last_state.h),1)
encoder_last_state = LSTMStateTuple(
    c=enc_last_state_c,
    h=enc_last_state_h
)

## Decoder: Attention

In [29]:
decoder_cell = LSTMCell(lstm_size_decoder)

In [30]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [31]:
decoder_lengths = encoder_inputs_length + 51

In [32]:
#weights
W = tf.Variable(tf.random_uniform([lstm_size_decoder, vocab_size], -1, 1), dtype=tf.float32)
#bias
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [33]:
assert EOS == 1 and PAD == 0

eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')

#Retrieves rows of the params tensor, using indexing with arrays in numpy
eos_step_embedded = tf.nn.embedding_lookup(input_embedding, eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(input_embedding, pad_time_slice)

In [34]:
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    #end of sentence
    initial_input = eos_step_embedded
    #last time steps cell state
    initial_cell_state = encoder_last_state
    #none
    initial_cell_output = None
    #decoder output as input => Teacher Enforcing needed
    initial_loop_state = GO  
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

In [35]:
# Attention Mechanism
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
    def get_next_input():
        #Dot product between previous ouput and weights, then + biases
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        #Returns the index with the largest value across axes of a tensor.
        prediction = tf.argmax(output_logits, axis=1)
        #embed prediction for the next input
        next_input = tf.nn.embedding_lookup(input_embedding, prediction)
        return next_input
    elements_finished = (time >= decoder_lengths) # this operation produces boolean tensor of [batch_size]
                                                  # defining if corresponding sequence has ended

    #Computes the "logical and" of elements across dimensions of a tensor.
    finished = tf.reduce_all(elements_finished) # -> boolean scalar
    #Return either fn1() or fn2() based on the boolean predicate pred.
    input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    
    #set previous to current
    state = previous_state
    output = previous_output + embedded_output[-1]
    loop_state = None

    return (elements_finished, 
            input,
            state,
            output,
            loop_state)

In [36]:
def loop_fn(time, previous_output, previous_state, previous_loop_state):
    if previous_state is None:    # time == 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

#ta = tensor array
decoder_outputs_ta, decoder_last_state, decoder_loop_state = tf.nn.raw_rnn(decoder_cell, loop_fn)
decoder_outputs = decoder_outputs_ta.stack()


In [37]:
decoder_outputs

<tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 256) dtype=float32>

In [38]:
#reduces dimensionality
decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
#flettened output tensor
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
#pass flattened tensor through decoder
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)
#prediction vals
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [39]:
#final prediction
decoder_prediction = tf.argmax(decoder_logits, 2)

## Optimizer

In [40]:
#cross entropy loss
#one hot encode the target values so we don't rank just differentiate
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)
#loss function
loss = tf.reduce_mean(stepwise_cross_entropy)
#train it 
train_op = tf.train.AdamOptimizer().minimize(loss)

In [41]:
type(x[0])

list

In [42]:
def next_feed():
    batch = x[:60]
    batch_y = y[:60]
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(batch)
    decoder_inputs_, _ = helpers.batch(
        [[GO] + (sequence) + [EOS] for sequence in batch_y]
    )
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] + [PAD] for sequence in batch_y]
    )
    
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_inputs: decoder_inputs_,
        decoder_targets: decoder_targets_,
    }

In [217]:
def next_feed_test():
    batch_x_test = x[53:60]
    dec_input = np.zeros((len(batch_x_test), 1))
    batch_y_test = y[53:60]
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(batch_x_test)
#     decoder_inputs_, _ = helpers.batch(
#         [[GO] + (sequence) + [EOS] + [PAD] * 3 for sequence in batch_y_test]
#     )
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] + [PAD] for sequence in batch_y_test]
    )
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_inputs: dec_input,
    }, batch_x_test, decoder_targets_

In [44]:
loss_track = []

In [45]:
save_every = 50
save_dir = 'data/'
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [46]:
max_batches = 12001
batches_in_epoch = 300
num_batch_save = 50

for batch in range(max_batches):
    fd = next_feed()
   
    _, batch_loss, batch_logits = sess.run([train_op, loss, decoder_logits], fd)
    if batch == 0 or batch % batches_in_epoch == 0:
        Epoch = batch // batches_in_epoch
        print('=============================EPOCH   :{:>4} ============================='.format(Epoch))
        print('batch {}/{}'.format(batch, max_batches))
        print('#####=====batch loss  :  {:>6.3f}'.format(batch_loss))
        predict_ = sess.run(decoder_prediction, fd)
        for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
            print('  sample {}:'.format(i + 1))
            print('    input     > {}'.format(inp))
            print('    prediction     > {}'.format(pred))
            if i >= 4:
                break
        accuracy = np.mean(batch_logits.argmax(axis=-1)==predict_)
        print('#####=====training accuracy  : {:>6.3f}'.format(accuracy))

=============================EPOCH   :   0 =============================
batch 0/12001
#####=====batch loss  :  12.787
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [565 565 565 565 565 565 565 565 565 565 565 565  94  94  94  94  94  94
  91  91 370 370 370 370 370 370  95  95  95  95  95  95  91 370  91  91
  91  91  91  91  91  91  91  91  95  95  91  95  95  95  95  95  95   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   

=============================EPOCH   :   2 =============================
batch 600/12001
#####=====batch loss  :   2.749
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    inp

=============================EPOCH   :   4 =============================
batch 1200/12001
#####=====batch loss  :   2.135
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

=============================EPOCH   :   6 =============================
batch 1800/12001
#####=====batch loss  :   1.558
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

=============================EPOCH   :   8 =============================
batch 2400/12001
#####=====batch loss  :   1.038
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

...

=============================EPOCH   :  24 =============================
batch 7200/12001
#####=====batch loss  :   0.117
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

=============================EPOCH   :  26 =============================
batch 7800/12001
#####=====batch loss  :   0.102
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

=============================EPOCH   :  28 =============================
batch 8400/12001
#####=====batch loss  :   0.086
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

=============================EPOCH   :  30 =============================
batch 9000/12001
#####=====batch loss  :   0.079
  sample 1:
    input     > [111  66   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
    prediction     > [60 82  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
  sample 2:
    in

KeyboardInterrupt: 

In [245]:
####==== Test Code
import nltk
_, source_batch, target_batch = next_feed_test()

dec_input = np.zeros((len(source_batch), 0))
for i in range(50):
    fd, _, _ = next_feed_test()
    batch_logits = sess.run([decoder_logits], fd)
    R = np.array(batch_logits)
    prediction = R.argmax(axis=-1)
    pred = prediction[0]
    pred_T = np.transpose(pred)
    dec_input = np.hstack([dec_input, pred[i][:,None]])
    dec_input_T = np.transpose(dec_input)
    print(dec_input_T)
    print(target_batch)
dec_input = np.transpose(dec_input_T)
dec_input_list = dec_input.tolist()
target_batch = np.transpose(target_batch)
target_batch_list = target_batch.tolist()
# print(dec_input_list)
# print(target_batch_list)

BLEUscore_0 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[0]], target_batch_list[0], weights = (0.5, 0.5))
print('#####=====BLEUscore_0  : {:>6.3f}'.format(BLEUscore_0))
BLEUscore_1 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[1]], target_batch_list[1], weights = (0.5, 0.5))
print('#####=====BLEUscore_1  : {:>6.3f}'.format(BLEUscore_1))
BLEUscore_2 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[2]], target_batch_list[2], weights = (0.5, 0.5))
print('#####=====BLEUscore_2  : {:>6.3f}'.format(BLEUscore_2))
BLEUscore_3 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[3]], target_batch_list[3], weights = (0.5, 0.5))
print('#####=====BLEUscore_3  : {:>6.3f}'.format(BLEUscore_3))
BLEUscore_4 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[4]], target_batch_list[4], weights = (0.5, 0.5))
print('#####=====BLEUscore_4  : {:>6.3f}'.format(BLEUscore_4))
BLEUscore_5 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[5]], target_batch_list[5], weights = (0.5, 0.5))
print('#####=====BLEUscore_5  : {:>6.3f}'.format(BLEUscore_5))
BLEUscore_6 = nltk.translate.bleu_score.sentence_bleu([dec_input_list[6]], target_batch_list[6], weights = (0.5, 0.5))
print('#####=====BLEUscore_6  : {:>6.3f}'.format(BLEUscore_6))

[[ 466.  466.  246.  448.  466.  448.  315.]]
[[466 466 246 448 466 448 315]
 [665 606 320 672 476 672 546]
 [348 318 166 184 426 640 123]
 [100 713  39 389 443 467  45]
 [166 420 160 262 648 707 294]
 [204 467 359 508 106 467   5]
 [298 562 142 634 570 137 108]
 [348 602 672 302 338 166 348]
 [585 409 348  74 383 280 338]
 [328 666 715 403 519 119 383]
 [142 280 602 467 348 353 276]
 [672 672 409 331 585 348 224]
 [468 201 335 348 335  92 519]
 [648  30 204 338 566 648 219]
 [ 24 335 386 712 131 331 290]
 [422 648 515   1 204 348 563]
 [348 653 350   0 579 338 166]
 [338 319 536   0 348 383 633]
 [383 467  74   0 449 131 552]
 [166 665 220   0 166 204 123]
 [348 348 348   0 348 579  45]
 [163 338 487   0 251 348 465]
 [696 712 459   0 613 187 508]
 [479   1 347   0 419 406 648]
 [368   0 161   0 335 142 646]
 [166   0   5   0 204 586 589]
 [417   0 684   0 556 348 335]
 [508   0 143   0 525 214  74]
 [302   0 648   0 488 308 106]
 [204   0 220   0 348 348 604]
 [353   0 348   0 167 64

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]]
[[466 466 246 448 466 448 315]
 [665 606 320 672 476 672 546]
 [348 318 166 184 426 640 123]
 [100 713  39 389 443 467  45]
 [166 420 160 262 648 707 294]
 [204 467 359 508 106 467   5]
 [298 562 142 634 570 137 108]
 [348 602 672 302 338 166 348]
 [585 409 348  74 383 280 338]
 [328 666 715 403 519 119 383]
 [142 280 602 467 348 353 276]
 [672 672 409 331 585 348 224]
 [468 201 335 348 335  92 519]
 [648  30 204 338 566 648 219]
 [ 24 335 386 712 131 331 290]
 [422 648 515   1 204 348 563]
 [348 653 350   0 579 338 166]
 [338 319 536   0 348 383 633]
 [383 467  74   0 449 131 552]
 [166 665 220   0 166 204 123]
 [348 348 348   0 348 579  45]
 [163 338 487   0 251 348 465]
 [696 712 459   0 613 187 508]
 [479   1 347   0 419 406 648]
 [368   0 161   0 335 142 646]

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]]
[[466 466 246 448 466 448 315]
 [665 606 320 672 476 672 546]
 [348 318 166 184 426 640 123]
 [100 713  39 389 443 467  45]
 [166 420 160 262 648 707 294]
 [204 467 359 508 106 467   5]
 [298 562 142 634 570 137 108]
 [348 602 672 302 338 166 348]
 [585 409 348  74 383 280 338]
 [328 666 715 403 519 119 383]
 [142 280 602 467 348 353 276]
 [672 672 409 331 585 348 224]
 [468 201 335 348 335  92 519]
 [648  30 204 338 566 648 219]
 [ 24 335 386 712 131 331 290]
 [422 648 515   1 204 348 563]
 [348 653 350   0 579 338 166]
 [338 319 536   0 348 383 633]
 [383 467  74   0 449 131 552]
 [166

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]]
[[466 466 246 448 466 448 315]
 [665 606 320 672 476 672 546]
 [348 318 166 184 426 640 123]
 [100 713  39 389 443 467  45]
 [166 420 160 262 648 707 294]
 [204 467 359 508 106 467   5]
 [298 562 142 634 570 137 108]
 [348 602 672 302 338 166 348]
 [585 409 348  74 383 280 338]
 [328 666 715 403 519 119 383]
 [142 280 602 467 348 353 276]
 [672 672 409 331 585 348 224]
 [468 201 335 348 335  92 519]
 [648  30 2

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]]
[[466 466 246 448 466 448 315]
 [665 606 320 672 476 672 546]
 [348 318 166 184 426 640 123]
 [100 713  39 389 443 467  45]
 [166 420 160 262 648 707 294]
 [204 467 359 508 106 467   5]
 [298 562 142 634 570 137 108]
 [348 602 672 302 338 166 348]
 [585 409 348  74 383 280 338]


...

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]
 [ 348.  653.  350.    0.  579.  338.  166.]
 [ 338.  319.  536.    0.  348.  383.  633.]
 [ 383.  467.   74.    0.  449.  131.  552.]
 [ 166.  665.  220.    0.  166.  204.  123.]
 [ 348.  348.  348.    0.  348.  579.   45.]
 [ 163.  338.  487.    0.  251.  348.  465.]
 [ 479.   

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]
 [ 348.  653.  350.    0.  579.  338.  166.]
 [ 338.  319.  536.    0.  348.  383.  633.]
 [ 383.  467.   74.    0.  449.  131.  552.]
 [ 166.  665.  220.    0.  166.  204.  123.]
 [ 348.  348.  348.    0.  348.  579.   45.]
 [ 163.  338.  487.    0.  251.  348.  465.]
 [ 479.   

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]
 [ 348.  653.  350.    0.  579.  338.  166.]
 [ 338.  319.  536.    0.  348.  383.  633.]
 [ 383.  467.   74.    0.  449.  131.  552.]
 [ 166.  665.  220.    0.  166.  204.  123.]
 [ 348.  348.  348.    0.  348.  579.   45.]
 [ 163.  338.  487.    0.  251.  348.  465.]
 [ 479.   

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]
 [ 348.  653.  350.    0.  579.  338.  166.]
 [ 338.  319.  536.    0.  348.  383.  633.]
 [ 383.  467.   74.    0.  449.  131.  552.]
 [ 166.  665.  220.    0.  166.  204.  123.]
 [ 348.  348.  348.    0.  348.  579.   45.]
 [ 163.  338.  487.    0.  251.  348.  465.]
 [ 479.   

[[ 466.  466.  246.  448.  466.  448.  315.]
 [ 665.  606.  320.  672.  476.  672.  546.]
 [ 348.  318.  166.  184.  426.  640.  123.]
 [ 100.  318.   39.  389.  443.  467.   45.]
 [ 166.  420.  160.  262.  648.  648.  294.]
 [ 204.  467.  359.  508.  106.  467.    5.]
 [ 298.  562.  142.  634.  570.  137.  108.]
 [ 348.  602.  672.  302.  338.  166.  348.]
 [ 585.  409.  348.   74.  383.  280.  338.]
 [ 328.  666.  602.  403.  519.  119.  383.]
 [ 142.  280.  602.  467.  348.  353.  276.]
 [ 672.  672.  409.  331.  585.  348.  224.]
 [ 468.  201.  335.  348.  335.   92.  519.]
 [ 648.   30.  204.  338.  566.  648.  219.]
 [  24.  335.  386.    1.  131.  331.  290.]
 [ 422.  648.  515.    1.  204.  348.  563.]
 [ 348.  653.  350.    0.  579.  338.  166.]
 [ 338.  319.  536.    0.  348.  383.  633.]
 [ 383.  467.   74.    0.  449.  131.  552.]
 [ 166.  665.  220.    0.  166.  204.  123.]
 [ 348.  348.  348.    0.  348.  579.   45.]
 [ 163.  338.  487.    0.  251.  348.  465.]
 [ 479.   

In [246]:
num_preds = 2
source_words = [[num2wordX[l] for l in sent if num2wordX[l]!=0] for sent in source_batch[:num_preds]]
dest_words = [[num2wordY[l] for l in sent] for sent in dec_input[:num_preds, 1:]]

for english, french in zip(source_words, dest_words):
    print('English======>',' '.join(english)+'\n'+'French ======>',' '.join(french),'\n')

English======> The Department's advocacy and special events programme is guided by the mandates emanating from the General Assembly and its various subsidiary bodies, and promotes the importance and significance of the Assembly's work in all of its facets.
French ======> activités de plaidoyer et le programme de manifestations spéciales du Département s'inspirent des mandats émanant de l'Assemblée générale et de ses organes organes subsidiaires, et visent à sensibiliser le public à l'importance et à la signification des travaux de l'Assemblée sous tous leurs aspects. Hammarskjöld 15 15 15 15 

English======> The weekly briefings for the non-governmental organization community affiliated with the Department frequently cover topics which refer to the activities of the General Assembly.
French ======> séances d'information d'information destinées aux organisations non gouvernementales affiliées au Département portent souvent sur des thèmes liés aux activités de l'Assemblée Hammarskjöld Ha